In [1]:
import cv2
import numpy as np
from random import randint
import math

from sklearn.ensemble import IsolationForest

In [67]:
img = cv2.imread('Pict_4_4.bmp')
verbose = True

In [68]:
def show(img):
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def sign(a):
    if a > 0:
        return 1
    elif a < 0:
        return -1
    return 0
def dist(a, b):
    return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5

def t_int(tup):
    return (int(tup[0]), int(tup[1]))

In [69]:
def getColor(img, r_base, g_base, b_base):
    b,g,r = cv2.split(img)
    b = b.astype(float)
    r = r.astype(float)
    g = g.astype(float)
    
    r_norm = max(abs(255 - r_base), r_base)**2
    g_norm = max(abs(255 - g_base), g_base)**2
    b_norm = max(abs(255 - b_base), b_base)**2
    
    dist = np.maximum((r - r_base)**2 / r_norm, 
                      (g - g_base)**2 / g_norm, 
                      (b - b_base)**2 / b_norm)
    dist /= dist.max()
    return (255 * (1 - dist)).astype(np.uint8)

# ТРИМИНОШКИ

In [70]:
b,g,r = cv2.split(img)
brown = getColor(img, 110, 70, 50)
if verbose:
    show(brown)

In [71]:
COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255), (255, 255, 0), (255, 0, 255), (255, 255, 255)]
class Triangle:
    def __init__(self, cnt):
                    
        left = tuple(cnt[cnt[:,:,0].argmin()][0])
        right = tuple(cnt[cnt[:,:,0].argmax()][0])
        top = tuple(cnt[cnt[:,:,1].argmin()][0])
        bottom = tuple(cnt[cnt[:,:,1].argmax()][0])

        distances = np.array([dist(left, top), dist(top, right), dist(right, bottom), dist(bottom, left)])

        self.points = [left, top, right, bottom]
        to_del = distances.argmin()
        self.points[(to_del + 1) % 4] = ((self.points[(to_del + 1) % 4][0] + self.points[to_del][0]) / 2, (self.points[(to_del + 1) % 4][1] + self.points[to_del][1]) / 2) 
        del self.points[distances.argmin()]

        self.minimum = distances.min()
        distances = np.delete(distances, [distances.argmin()])
        self.mean = distances.mean()
        self.variance = ((distances - distances.mean())**2).mean()**0.5
        self.sureness = 1
        
        self.center = ((self.points[0][0] + self.points[1][0] + self.points[2][0]) / 3, 
                       (self.points[0][1] + self.points[1][1] + self.points[2][1]) / 3)
        self.x_min = int(min(self.points[0][0], self.points[1][0], self.points[2][0]))
        self.x_max = int(max(self.points[0][0], self.points[1][0], self.points[2][0]))
        self.y_min = int(min(self.points[0][1], self.points[1][1], self.points[2][1]))
        self.y_max = int(max(self.points[0][1], self.points[1][1], self.points[2][1]))
        
    def sameSide(self, a, b, c):
        return sign((b[0] - a[0]) * (c[1] - a[1]) - (b[1] - a[1]) * (c[0] - a[0]))
    
    def contains(self, A):
        return (self.sameSide(self.points[0], self.points[1], A) == self.sameSide(self.points[0], self.points[1], self.points[2])
            and self.sameSide(self.points[1], self.points[2], A) == self.sameSide(self.points[1], self.points[2], self.points[0])
            and self.sameSide(self.points[2], self.points[0], A) == self.sameSide(self.points[2], self.points[0], self.points[1]))
    
    def add(self, T):
        self.sureness += T.sureness
        
        V = self.variance + T.variance
        self.points = [((self.points[0][0] * T.variance + T.points[0][0] * self.variance) / V, (self.points[0][1] * T.variance + T.points[0][1] * self.variance) / V),
                       ((self.points[1][0] * T.variance + T.points[1][0] * self.variance) / V, (self.points[1][1] * T.variance + T.points[1][1] * self.variance) / V),
                       ((self.points[2][0] * T.variance + T.points[2][0] * self.variance) / V, (self.points[2][1] * T.variance + T.points[2][1] * self.variance) / V)]
        
        distances = np.array([dist(self.points[0], self.points[1]), dist(self.points[1], self.points[2]), dist(self.points[0], self.points[2])])
        self.variance = (self.variance * T.variance) / (self.variance + T.variance)
        
        
    def draw(self, image):
        n = randint(0, 6)
        cv2.line(image, t_int(self.points[0]), t_int(self.points[1]), COLORS[n], 2)
        cv2.line(image, t_int(self.points[1]), t_int(self.points[2]), COLORS[n], 2)
        cv2.line(image, t_int(self.points[0]), t_int(self.points[2]), COLORS[n], 2)
        cv2.circle(image, t_int(self.center), 5, COLORS[n], -1)

In [72]:
ans = []

for mode in [True, False]:
    for thres in range(5, 255, 5):        
        for orig in [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), r, g, b, brown]:
            
            if mode:
                dil = cv2.dilate(orig, np.ones((3, 3)), iterations=5)
                orig = cv2.subtract(dil, orig)
            else:
                er = cv2.dilate(orig, np.ones((3, 3)), iterations=5)
                orig = cv2.subtract(orig, er)
            
            orig = cv2.threshold(orig, thres, 255, cv2.THRESH_BINARY)[1]

            im2, contours, hierarchy = cv2.findContours(orig, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

            for i, cnt in enumerate(contours):
                if cv2.contourArea(cnt) > 1000:
                    T = Triangle(cnt)

                    if T.minimum < 10 and T.mean > 60 and T.mean < 200 and T.variance / T.mean < 0.06:
                        ans.append(T)

if verbose:
    img2 = img.copy()
    for T in ans:
        T.draw(img2)
    show(img2)

In [73]:
T_sizes = np.array([T.mean for T in ans])
best_size = np.median(T_sizes)

In [74]:
final_ans = []
for T in ans:
    if abs(T.mean - best_size) / best_size < 0.15:
        new_one = True

        for i, True_T in enumerate(final_ans):
            if True_T.contains(T.center) or T.contains(True_T.center):
                final_ans[i].add(T)
                new_one = False
                break
                
        if new_one:
            final_ans.append(T)
if verbose:            
    img2 = img.copy()
    for T in final_ans:
        T.draw(img2)
    show(img2)

# ТОЧКИ

In [75]:
class Circle:
    def __init__(self, center, radius, area, iter):
        self.center = (int(center[0]), int(center[1]))
        self.radius = radius
        self.area = area
        
        self.sureness = 1
        self.algorithms = {iter}
        
    def draw(self, image):
        n = randint(0, 6)
        cv2.circle(image, t_int(self.center), int(self.radius), COLORS[n], 1)
        
    def add(self, c):
        self.algorithms = self.algorithms.union(c.algorithms)
        V = self.sureness + c.sureness
        self.center = ((self.center[0] * self.sureness + c.center[0] * c.sureness) / V, (self.center[1] * self.sureness + c.center[1] * c.sureness) / V)
        self.radius = (self.radius * self.sureness + c.radius * c.sureness) / V
        self.sureness = V        

In [76]:
for T in final_ans:
    T.circles = []
    T.colors = np.empty((0, 3))
    for x in range(T.x_min, T.x_max):
        for y in range(T.y_min, T.y_max):        
            if T.contains((x, y)):
                T.colors = np.vstack([T.colors, img[y, x]])
if verbose:
    print("masks generated")
                
for iter, subset in enumerate([[0], [1], [2], [0, 1], [0, 2], [1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]):
    for j, T in enumerate(final_ans):
        IForest = IsolationForest(n_estimators=100).fit(T.colors[:, subset])

        roi = img[T.y_min:T.y_max, T.x_min:T.x_max, subset]

        mask = IForest.decision_function(roi.reshape(roi.shape[0] * roi.shape[1], -1)).reshape(roi.shape[0], roi.shape[1])
        mask = (255 - ((mask - mask.min()) / (mask.max() - mask.min()) * 255)).astype(np.uint8)
        
        for thr in range(60, 150, 5):
            new_mask = (255 * (cv2.medianBlur(mask, ksize = 3) > thr)).astype(np.uint8)
            
            im2, contours, hierarchy = cv2.findContours(new_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

            for i, cnt in enumerate(contours):
                (x,y),radius = cv2.minEnclosingCircle(cnt)

                if (cv2.contourArea(cnt) > best_size**2 / 500 and cv2.contourArea(cnt) < 100 and 
                    cv2.contourArea(cnt) / (math.pi * radius**2) > 0.5 and
                    T.contains((x + T.x_min, y + T.y_min))):
                    T.circles.append(Circle((x + T.x_min, y + T.y_min), radius, cv2.contourArea(cnt), iter))
    
    if verbose:
        print("|", end="")

if verbose:
    img2 = img.copy()
    for T in final_ans:
        for C in T.circles:
            C.draw(img2)
    show(img2)

masks generated
|||||||||

In [77]:
C_sizes = np.array([c.area for T in final_ans for c in T.circles])
best_area = np.median(C_sizes)

In [78]:
for T in final_ans:
    T.final_circles = []
    for c in T.circles:
        if abs(c.area - best_area) / best_area < 0.6:
            new_one = True

            for i, True_c in enumerate(T.final_circles):
                if dist(True_c.center, c.center) <= max(True_c.radius, c.radius):
                    T.final_circles[i].add(c)
                    new_one = False
                    break

            if new_one:
                T.final_circles.append(c)

if verbose:
    img2 = img.copy()
    for T in final_ans:
        for C in T.final_circles:
            C.draw(img2)
    show(img2)

In [79]:
for T in final_ans:
    final_circles = []
    for c in T.final_circles:
        if len(c.algorithms) > 4:
            final_circles.append(c)
    T.final_circles = final_circles.copy()

if verbose:
    img2 = img.copy()
    for T in final_ans:
        for C in T.final_circles:
            C.draw(img2)
    show(img2)

In [80]:
for T in final_ans:
    T.markers = [0, 0, 0]
    for c in T.final_circles:
        dists = np.array([dist(c.center, T.points[i]) for i in range(3)])
        T.markers[dists.argmin()] += 1

In [81]:
f = open('answer.txt', 'w')
f.write(str(len(final_ans)) + "\n")
for T in final_ans:
    f.write(str(int(T.center[0])) + " " + 
            str(int(T.center[1])) + " " + 
            str(T.markers[0]) + " " + 
            str(T.markers[1]) + " " + 
            str(T.markers[2]) + "\n")
f.close()

In [82]:
img2 = img.copy()
for T in final_ans:
    font = cv2.FONT_HERSHEY_SIMPLEX    
    cv2.putText(img2, str(T.markers[0]) + " " + 
            str(T.markers[1]) + " " + 
            str(T.markers[2]), t_int(T.center), font, 0.75, (255, 255, 255), 2)
if verbose:
    show(img2)